In [2]:
import requests
import json
import pandas as pd

hashtag_name = '친환경세제'
url_first = 'https://www.instagram.com/explore/tags/{}/'.format(hashtag_name)
sess = requests.Session()
response = sess.get(url_first)
result = json.loads(response.text)

def first_page():
    post_list = []
    end_point = result['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
    edges = result['graphql']['hashtag']['edge_hashtag_to_media']['edges']
    for n in range(len(edges)):
        try:
            text = result['graphql']['hashtag']['edge_hashtag_to_media']['edges'][n]['node']['edge_media_to_caption']['edges'][0]['node']['text']
        except:
            text = None
        like = result['graphql']['hashtag']['edge_hashtag_to_media']['edges'][n]['node']['edge_liked_by']['count']
        img_url = result['graphql']['hashtag']['edge_hashtag_to_media']['edges'][n]['node']['thumbnail_src']
        post = {'text': text, 'like': like, 'img_url': img_url}
        post_list.append(post)
    return post_list, end_point

def crawl_page(post_list, end_point):
    while end_point != None:
        url = 'https://www.instagram.com/graphql/query/?query_hash=90cba7a4c91000cf16207e4f3bee2fa2&variables={\"tag_name\":\"%s\",\"first\":1,\"after\":\"%s\"}' % (
            hashtag_name, end_point)
        sess = requests.Session()
        response = sess.get(url)
        result = json.loads(response.text)
        end_point = result['data']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
        print(end_point)
        edges = result['data']['hashtag']['edge_hashtag_to_media']['edges']
        for i in range(len(edges)):
            try:
                text = result['data']['hashtag']['edge_hashtag_to_media']['edges'][i]['node']['edge_media_to_caption'][
                    'edges'][0]['node']['text']
            except:
                text = None
            like = result['data']['hashtag']['edge_hashtag_to_media']['edges'][i]['node']['edge_liked_by']['count']
            img_url = result['data']['hashtag']['edge_hashtag_to_media']['edges'][i]['node']['thumbnail_src']
            post = {'text': text, 'like': like, 'img_url': img_url}
            print(post)
            post_list.append(post)
    return post_list

post_list, end_point = first_page()
post_list = crawl_page(post_list, end_point)

final = pd.DataFrame(post_list)
final.to_csv("/Users/hyerin/PycharmProjects/GDT_study/filnalProject/instagramData/crawling02.csv", encoding='utf-8-sig')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)